# Github top 256 contributors' stars

Récupérer via crawling la liste des 256 top contributors sur cette page https://gist.github.com/paulmillr/2657075
En utilisant l'API github (https://developer.github.com/v3/) récupérer pour chacun de ces users le nombre moyens de stars des repositories qui leur appartiennent. Pour finir, classer ces 256 contributors par leur note moyenne.﻿

Comme l'API github dispose de restrictions d'accès (limitation du nombre de requêtes), vous aurez besoin de vous authentifier via un token: https://help.github.com/en/articles/creating-a-personal-access-token-for-the-command-line.

In [1]:
import asyncio
from multiprocessing import Pool
from pyppeteer import launch # conda install -c conda-forge pyppeteer
from github import Github # conda install -c conda-forge pygithub
from functools import reduce

In [2]:
ghub = Github('4b58b591b43e18a6264f943d3137c3e24ef9f176')

## Scrap top 256 users

From https://gist.github.com/paulmillr/2657075

```scala
githubUsers
  .filter(user => user.followers > 1000)
  .sortBy('contributions')
  .slice(0, 256)
```

In [3]:
browser = await launch()
page = await browser.newPage()
await page.goto('https://gist.github.com/paulmillr/2657075')
contribTable = await page.xpath('//*[@id="file-active-md-readme"]/article/table')
contributors  = await contribTable[0].JJeval('a', '(nodes => nodes.map(n => n.innerText))')

In [4]:
# To get contributors URI, better than innerText
#contributors = await page.xpath('//*[@id="file-active-md-readme"]/article/table/tbody/tr/td/a')
# handle = await contributors[0].getProperty('href')
# href = await handle.jsonValue()

In [5]:
# Remove empty contributor names
contributors2 = list(filter(lambda c: len(c) > 0, contributors))

## For each user get the number of stars for owned repositories

Save average number of stars

In [6]:
def getAverageStars(contributor):
    try:
        repos = ghub.get_user(contributor).get_repos()
        stargazers_counts = map(lambda r: r.stargazers_count, repos)
        if stargazers_counts:
            starCount = reduce(lambda a, b: a + b, stargazers_counts)
            starAverage = starCount / repos.totalCount
        else:
            starAverage = 0.0
    except:
        starAverage = -1.0
    print(contributor, starAverage)
    return (contributor, starAverage)

In [7]:
# !!Triggering abuse detection if number worker > 1
workers = Pool(1)
stars2 = workers.map(getAverageStars, contributors2)

fabpot 25.275
andrew 3.7408906882591095
taylorotwell 8.333333333333334
egoist 57.41863905325444
HugoGiraudel 173.30769230769232
ornicar 25.344213649851632
bebraw 35.9625
nelsonic 2.0412371134020617
alexcrichton 16.26525198938992
jonathanong 12.779411764705882
mikermcneil 3.6862745098039214
benbalter 27.741784037558684
jxnblk 115.21189591078067
yegor256 53.71578947368421
orta 12.496433666191155
rstacruz 106.37821782178217
GrahamCampbell 135.17241379310346
afc163 7.1647727272727275
kamranahmedse 2454.438596491228
joshaber 64.74468085106383
bkeepers 46.372093023255815
kennethreitz 376.0
kytrinyx 5.741379310344827
STRML 45.63352272727273
atmos 20.416666666666668
weierophinney 3.4115384615384614
agentzh 26.6875
steipete 401.10869565217394
ai 382.7169811320755
mikepenz 419.0
nvie 197.59701492537314
hadley 53.674107142857146
appleboy 18.870967741935484
Rich-Harris 46.02797202797203
drnic 7.810671256454389
rafaelfranca 5.606451612903226
Ocramius 29.117857142857144
mitchellh 186.77777777777777


In [8]:
len(stars2)

257

In [16]:
from operator import itemgetter
sorted(stars2, reverse=True, key=itemgetter(1))

[('bailicangdu', 3253.9473684210525),
 ('vinta', 3213.875),
 ('kamranahmedse', 2454.438596491228),
 ('mrdoob', 2089.71875),
 ('nolimits4web', 1853.1666666666667),
 ('sebastianbergmann', 1814.140625),
 ('junyanz', 1811.0),
 ('ruanyf', 1507.8196721311476),
 ('IanLunn', 1497.888888888889),
 ('gorhill', 1225.7142857142858),
 ('JakeWharton', 1093.8796296296296),
 ('Unknwon', 902.7631578947369),
 ('binux', 887.3636363636364),
 ('photonstorm', 884.5714285714286),
 ('jaywcjlove', 754.4875),
 ('mdo', 723.4814814814815),
 ('connors', 685.1428571428571),
 ('fzaninotto', 626.0888888888888),
 ('amitshekhariitbhu', 572.0540540540541),
 ('dennybritz', 549.188679245283),
 ('krzysztofzablocki', 514.925),
 ('ded', 448.6923076923077),
 ('mikepenz', 419.0),
 ('steipete', 401.10869565217394),
 ('LeaVerou', 399.04761904761904),
 ('anishathalye', 394.57894736842104),
 ('daylerees', 387.1515151515151),
 ('tj', 384.0642857142857),
 ('ai', 382.7169811320755),
 ('zenorocha', 380.6551724137931),
 ('JohnSundell', 